In [1]:
# General imports
import numpy as np

# Qiskit ansatz circuits
from qiskit.circuit.library import EfficientSU2, TwoLocal

# Qiskit primitives
from qiskit.primitives import Estimator as QiskitEstimator
from qiskit.primitives import Sampler as QiskitSampler

# Qiskit runtime
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Estimator, Sampler, Session

# Qiskit optimization
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.workflows import QUBO_transformer
from qiskit_optimization.passes.qubo_to_ising import QUBO2Ising
from qiskit_optimization.passes.eval_solution import EvaluateProgramSolution
from qiskit_optimization.passes.qubo_unroller import UnrollQUBOVariables

# Docplex
from docplex.mp.model import Model

# SPSA
from qiskit_optimization.spsa import minimize_spsa

## Load the Runtime (if using)

In [2]:
#service = QiskitRuntimeService()

In [3]:
#backend = service.get_backend('ibm_nazca')

## Build optimization problem using docplex
Define problem using standard classical representation. **This is the classical input.  Nothing quantum is here.**

In [4]:
mdl = Model("docplex model")
x = mdl.binary_var("x")
y = mdl.integer_var(lb=-2, ub=2, name="y")
mdl.minimize(2 * y - x)
# Min-result should be x = 1, y = -2 => -5
#print(mdl.export_as_lp_string())

## Convert to our `QuadraticProgram` format

In [5]:
qp = QuadraticProgram.from_docplex_mp(mdl)

## Perform transformations to QUBO and return new program
**Nothing quantum happens here either**

In [6]:
qubo_transformer = QUBO_transformer()
qubo = qubo_transformer.run(qp)

## All the quantum stuff is here ⚛️
### Convert from QUBO to Ising

In [7]:
hamiltonian = QUBO2Ising().run(qubo)

### Standard cost function definition

In [8]:
def cost_func(params, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (Estimator): Estimator primitive instance

    Returns:
        float: Energy estimate
    """
    cost = estimator.run(ansatz, hamiltonian, parameter_values=params).result().values[0]
    return cost

### Setup estimator and sampler instances

In [9]:
#session = Session(backend=backend)
#estimator = Estimator(session=session, options={"shots": int(1e4)})
#sampler = Sampler(session=session, options={"shots": int(1e4)})
estimator = QiskitEstimator(options={"shots": int(1e4)})
sampler = QiskitSampler(options={"shots": int(1e4)})

### Perform minimization

In [18]:
ansatz = TwoLocal(hamiltonian.num_qubits, 'ry', 'cx', 'linear', reps=1)
x0 = 2*np.pi*np.random.random(size=ansatz.num_parameters)
res = minimize_spsa(cost_func, x0, args=(ansatz, hamiltonian, estimator), maxiter=100)

### Computute distribution at found minimum

In [19]:
# Assign solution parameters to ansatz
qc = ansatz.assign_parameters(res.x)
qc.measure_all()
samp_dist = sampler.run(qc, shots=int(1e4)).result().quasi_dists[0]
# Close the session since we are now done with it
#session.close()

### Express quantum solution as QUBO variable values
Convert bit-string solutions to variable values in QUBO problem

In [20]:
res = EvaluateProgramSolution(qubo).run(samp_dist)
res

(-5.0, array([1, 0, 0, 0]))

## Classical postprocess
Unrolling QUBO variables back to the original problem definition. **This is classical processing. Nothing quantum happens here**

In [22]:
solution = UnrollQUBOVariables(qubo_transformer).run(res)
solution

array([ 1., -2.])

## Intepretation of solution
Depends on input, but is completely separate from quantum having done the processing for the solution.